In [3]:
import yaml
import json
from openai import OpenAI
import sqlite3
import pandas as pd

In [4]:
with open('api_keys.yaml', 'r') as f:
    api_keys = yaml.safe_load(f)

OPENAI_API_KEY = api_keys['openai-key']
client = OpenAI(api_key=OPENAI_API_KEY)

## Define tools and functions
Need functions to determine what tables to call. Predefined table structure

In [5]:
db_name = "lldm.db"
try: 
    db = sqlite3.connect(db_name) 
    print('Connected to DB')
except: 
    print("DB connection failed")

Connected to DB


In [6]:
# run query and return pandas dataframe
def run_query(db, query):
    try:
        df = pd.read_sql_query(query, db)
        return df
    # not a select statement
    except TypeError:
        cursor = db.cursor()
        cursor.execute(query)
        return    

In [7]:
sql_query = """
SELECT name FROM sqlite_master  
WHERE type='table';
"""
cursor = db.cursor()
cursor.execute(sql_query)
print(cursor.fetchall())

[('CHARACTER_SHEET',), ('INVENTORY',), ('SETTINGS',), ('NPCS',), ('TREASURES',), ('MONSTERS',), ('PLOT',), ('LOGS',), ('CAMPAIGN',), ('CHARACTER_INVENTORY',)]


In [8]:
query = '''
SELECT * FROM INVENTORY;
'''
run_query(db,query)

,Item_ID,Category,Item,Description
0,0,Weapon,Longsword,Elara's main weapon for combat.
1,1,Weapon,Dagger,Versatile tool for close combat and utility pu...
2,2,Adventuring Gear,Backpack,To carry essential items.
3,3,Adventuring Gear,Rope (50 feet),Useful for climbing and securing objects.
4,4,Adventuring Gear,Rations (5 days),Enough food for the journey.
5,5,Adventuring Gear,Water skin,To carry water.
6,6,Adventuring Gear,Flint and Steel,For starting fires.
7,7,Adventuring Gear,Healing Potions (2),For quick recovery in emergencies.
8,8,Armor,Chain Mail,Provides solid protection while allowing mobil...
9,9,Armor,Shield,Additional defense against attacks.


In [9]:
query = '''
SELECT * FROM CHARACTER_INVENTORY;
'''
run_query(db,query)

,Campaign_ID,Character_ID,Item_ID,Quantity


### Picking up item
For now, this is the pipeline:
- Item must come from INVENTORY table in db
- Assume that the item will always be written exactly as it appears in the table
- 

In [26]:
# check if item queried is in INVENTORY table
def validate_item(item_name):
    query = f'''
    SELECT * FROM INVENTORY WHERE UPPER(Item) LIKE "{item_name.upper()}%" 
    '''
    tmp = run_query(db, query)
    if not tmp.empty:
        return tmp['Item_ID'].iloc[0]
    return None

# update table if item validated, otherwise error message
# for now, use temporary campaign and character id
def get_obtained_item(item_name, quantity, campaign_id=0, character_id=0):
    item_id = validate_item(item_name)
    print(item_id, quantity)
    if item_id is not None:
        # TODO: error handling
        cursor = db.cursor()
        query = f'''
        UPDATE CHARACTER_INVENTORY SET Quantity = Quantity + {quantity}
        WHERE Item_ID = {item_id} AND Campaign_ID = {campaign_id} AND Character_ID = {character_id}
        '''
        cursor.execute(query)
        if cursor.rowcount == 0:
            query = f'''
            INSERT INTO CHARACTER_INVENTORY (Campaign_ID, Character_ID, Item_ID, Quantity) VALUES ({campaign_id}, {character_id}, {item_id}, {quantity})
            '''
            cursor.execute(query)
        return json.dumps({'message':'The item(s) were successfully obtained. Please continue the story.'})
    else:
        return json.dumps({'message':'Item does not exist. Please prompt user to specify further or provide another action.'})


def run_conversation_obtain_item(query):
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": query}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_obtained_item",
                "description": "Extract the item that the user has obtained in some manner (such as picked up, purchased, etc.)",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "item_name": {
                            "type": "string",
                            "description": "The name of the obtained item.",
                        },
                        "quantity": {
                            "type": "integer",
                            "description": "The number of said items obtained. If a number is not specified, try and infer based on the surrounding context."
                        }
                    },
                    "required": ["item_name", "quantity"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        #tool_choice="auto",  # auto is default, but we'll be explicit
        tool_choice = {"type":'function','function':{'name':'get_obtained_item'}}
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_obtained_item": get_obtained_item,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                item_name=function_args.get("item_name"),
                quantity=function_args.get("quantity"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response


In [27]:
print(run_conversation_obtain_item("I pick up a longsword from the bodies of the dead goblins."))

0 1
ChatCompletion(id='chatcmpl-9fc0r6t4YKcP2uX9X3UOB0IAMX0DY', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='You pick up the longsword from the bodies of the dead goblins. It has a sturdy blade and the hilt is wrapped in worn leather. Feeling a surge of confidence with this new weapon in hand, you look around and decide your next course of action.\n\nYou notice the cave extends deeper to the north, while to the south lies the flickering light of the outside world. Pathways to the east and west are also visible, each shrouded in darkness.\n\nWhat do you do next?\n- Head north, deeper into the cave.\n- Head south, back towards the cave entrance.\n- Explore the path to the east.\n- Explore the path to the west.', role='assistant', function_call=None, tool_calls=None))], created=1719704597, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_d576307f90', usage=CompletionUsage(completion_token

In [28]:
# Check to see if tables were updated
# TODO: WHY IS THIS UPDATED TWICE??
query = '''
SELECT * FROM CHARACTER_INVENTORY;
'''
run_query(db,query)

,Campaign_ID,Character_ID,Item_ID,Quantity
0,0,0,0,3.0


In [53]:
query = '''
DELETE FROM CHARACTER_INVENTORY;
'''
run_query(db,query)

### Discarding item
For now, this is the pipeline:
- Item must come from INVENTORY table in db AND must exist in CHARACTER_INVENTORY
- Assume that the item will always be written exactly as it appears in the table

In [37]:
# defining custom exceptions to be used with error handling
class ItemNotFoundException(Exception):
    pass

class ItemNotPossessedException(Exception):
    pass

In [40]:
# check if item queried is in INVENTORY table and character has more than discard amount
# might need a third error condition if item exists, user has item, but has less than discard amount
def validate_item_discard(item_name, quantity, campaign_id, character_id):
    query = f'''
    SELECT * FROM INVENTORY WHERE UPPER(Item) LIKE "{item_name.upper()}%" 
    '''
    tmp = run_query(db, query)
    if not tmp.empty: # item exists in world
        item_id = tmp['Item_ID'].iloc[0]
        # validate if character has item
        query = f'''
        SELECT * FROM CHARACTER_INVENTORY 
        WHERE Campaign_ID = {campaign_id} AND Character_ID = {character_id} AND Item_ID = {item_id} AND Quantity >= {quantity}
        '''
        # there should theoretically only be one row of the item for each character with the quantity as different values
        # need to validate and put checks in place
        tmp = run_query(db, query)
        if not tmp.empty:
            return item_id
        raise ItemNotPossessedException("Character does not have the item in their inventory.")
    raise ItemNotFoundException("Item does not exist in this campaign.")

# update table if item validated, otherwise error message
# for now, use temporary campaign and character id
# first update with item subtraction, then remove all entries with <=0 values
def get_discarded_item(item_name, quantity, campaign_id=0, character_id=0):
    try:
        item_id = validate_item(item_name)
        print(item_id, quantity)
        cursor = db.cursor()
        query = f'''
        UPDATE CHARACTER_INVENTORY SET Quantity = Quantity - {quantity}
        WHERE Item_ID = {item_id} AND Campaign_ID = {campaign_id} AND Character_ID = {character_id};
        '''
        cursor.execute(query)

        # housekeeping query, remove rows with invalid quantites (<= 0 items)
        query = 'DELETE FROM CHARACTER_INVENTORY WHERE Quantity <= 0'
        cursor.execute(query)
        return json.dumps({'message':'The item(s) were successfully discarded. Please continue the story.'})
    except ItemNotPossessedException as e:
        return json.dumps({'message':"Item is not in character's possession. Please prompt user to specify further or provide another action."})
    except ItemNotFoundException as e:
        return json.dumps({'message':"Item does not exist. Please prompt user to specify further or provide another action."})


def run_conversation_discard_item(query):
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": query}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_discarded_item",
                "description": "Extract the item that the user has discarded in some manner (such as thrown away, consumed, broken, etc.)",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "item_name": {
                            "type": "string",
                            "description": "The name of the discarded item.",
                        },
                        "quantity": {
                            "type": "integer",
                            "description": "The number of said items discarded. If a number is not specified, try and infer based on the surrounding context."
                        }
                    },
                    "required": ["item_name", "quantity"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
        # tool_choice = {"type":'function','function':{'name':'get_discarded_item'}}
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_discarded_item": get_discarded_item,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            print(function_args.get("item_name"), function_args.get("quantity"))
            function_response = function_to_call(
                item_name=function_args.get("item_name"),
                quantity=function_args.get("quantity"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response


In [41]:
print(run_conversation_discard_item("I throw my longsword off the cliff overlooking the sea."))

longsword 1
0 1
ChatCompletion(id='chatcmpl-9fcesxfDTpa3Fl0DvFTa7j6z9p4Fu', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Your longsword sails through the air, glittering in the sunlight before it splashes into the churning sea below. The waves quickly engulf it, leaving no trace of your weapon. The cliffside now feels oddly barren without the familiar weight at your side. What do you do next?', role='assistant', function_call=None, tool_calls=None))], created=1719707078, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_ce0793330f', usage=CompletionUsage(completion_tokens=58, prompt_tokens=72, total_tokens=130))


In [43]:
# Check to see if tables were updated
query = '''
SELECT * FROM CHARACTER_INVENTORY;
'''
run_query(db,query)

,Campaign_ID,Character_ID,Item_ID,Quantity
0,0,0,0,4.0


### Combine the two functions

In [44]:
def run_conversation_item(query):
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": query}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_obtained_item",
                "description": "Extract the item that the user has obtained in some manner (such as picked up, purchased, etc.)",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "item_name": {
                            "type": "string",
                            "description": "The name of the obtained item.",
                        },
                        "quantity": {
                            "type": "integer",
                            "description": "The number of said items obtained. If a number is not specified, try and infer based on the surrounding context."
                        }
                    },
                    "required": ["item_name", "quantity"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "get_discarded_item",
                "description": "Extract the item that the user has discarded in some manner (such as thrown away, consumed, broken, etc.)",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "item_name": {
                            "type": "string",
                            "description": "The name of the discarded item.",
                        },
                        "quantity": {
                            "type": "integer",
                            "description": "The number of said items discarded. If a number is not specified, try and infer based on the surrounding context."
                        }
                    },
                    "required": ["item_name", "quantity"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
        # tool_choice = {"type":'function','function':{'name':'get_discarded_item'}}
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_obtained_item": get_obtained_item,
            "get_discarded_item": get_discarded_item,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            print(function_args.get("item_name"), function_args.get("quantity"))
            function_response = function_to_call(
                item_name=function_args.get("item_name"),
                quantity=function_args.get("quantity"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response

In [45]:
print(run_conversation_item("I pick up a longsword from the bodies of the dead goblins."))

longsword 1
0 1
ChatCompletion(id='chatcmpl-9feodNU7QT8A90NQ6wFIBheWRoEC8', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='You pick up the longsword from the bodies of the dead goblins. The blade is slightly chipped and tarnished, but it’s still a functional weapon. You feel its weight in your hand, a reliable addition to your arsenal.\n\nAs you stand up, you notice the path ahead branches into two directions: one leads deeper into the darker sections of the cave, where you can hear a faint dripping sound and an occasional growl; the other path seems to rise slightly, leading towards an exit with faint sunlight seeping through.\n\nWhat would you like to do next?\n- Explore the deeper sections of the cave.\n- Head towards the exit.\n- Examine the area for more items.\n- Something else. (Specify)\n\n', role='assistant', function_call=None, tool_calls=None))], created=1719715371, model='gpt-4o-2024-05-13', object='chat.completion', ser

In [46]:
# Check to see if tables were updated
query = '''
SELECT * FROM CHARACTER_INVENTORY;
'''
run_query(db,query)

,Campaign_ID,Character_ID,Item_ID,Quantity
0,0,0,0,5.0


In [47]:
print(run_conversation_item("I throw my longsword off the cliff overlooking the sea."))

longsword 1
0 1
ChatCompletion(id='chatcmpl-9feooyf3E0Pz3s1pSI3Z5LgQ1eod0', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='You watch as your longsword spins through the air, glinting in the sunlight before disappearing into the waves far below. The roar of the ocean fills your ears as the weapon vanishes from sight. \n\nWhat would you like to do next?', role='assistant', function_call=None, tool_calls=None))], created=1719715382, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_ce0793330f', usage=CompletionUsage(completion_tokens=50, prompt_tokens=72, total_tokens=122))


In [48]:
# Check to see if tables were updated
query = '''
SELECT * FROM CHARACTER_INVENTORY;
'''
run_query(db,query)

,Campaign_ID,Character_ID,Item_ID,Quantity
0,0,0,0,4.0
